 # Applying the water balance equation



 ## Objectives

 - Apply the water balance equation to real-world data

 - Test the impact of land cover on the runoff ratio

 - Fit a simple statistical model to data



 ## Prerequisites:

 - Basic understanding of Python

 - Familiarity with Pandas, Matplotlib

 ## Dataset

 We will be using the CAMELS-GB dataset. This contains daily hydrometeorological data for around 670 catchments in Great Britain, as well as catchment attributes related to land use/land cover, geology, and climate. Download the data [here](https://catalogue.ceh.ac.uk/documents/8344e4f3-d2ea-44f5-8afa-86d2987543a9), then upload it to the (currently) empty `data` folder in this repository. Now extract the data archive: 

In [67]:
import os
import zipfile

zip_path = 'data/8344e4f3-d2ea-44f5-8afa-86d2987543a9.zip'
extract_dir = 'data/8344e4f3-d2ea-44f5-8afa-86d2987543a9'

if not os.path.isdir(extract_dir):
    try:
        with zipfile.ZipFile(zip_path, 'r') as archive:
            archive.extractall('data')
    except FileNotFoundError:
        print(f"Zip file {zip_path} not found.")
    except zipfile.BadZipFile:
        print(f"Zip file {zip_path} is corrupt.")

Let's create a path variable so that we can easily navigate to the data files:

In [68]:
import os
DATADIR = os.path.join('data', '8344e4f3-d2ea-44f5-8afa-86d2987543a9', 'data')


 Now Load the data for a catchment chosen at random. The timeseries data are stored as csv files, so we use Pandas to load them into a Pandas DataFrame object:

In [69]:
import pandas as pd
id = '97002'
data = pd.read_csv(os.path.join(DATADIR, 'timeseries', f'CAMELS_GB_hydromet_timeseries_{id}_19701001-20150930.csv'), parse_dates=[0])
data.head()


,date,precipitation,pet,temperature,discharge_spec,discharge_vol,peti,humidity,shortwave_rad,longwave_rad,windspeed
0,1970-10-01,9.93,1.02,8.91,NaN,NaN,1.31,6.01,61.76,325.33,7.65
1,1970-10-02,4.01,1.41,7.66,NaN,NaN,1.76,5.11,93.56,294.20,10.03
2,1970-10-03,7.27,1.17,8.77,NaN,NaN,1.40,5.41,61.95,321.14,5.41
3,1970-10-04,3.77,0.06,9.74,NaN,NaN,0.23,7.76,42.83,341.28,7.27
4,1970-10-05,1.19,1.56,9.46,NaN,NaN,1.86,5.49,92.13,299.08,7.90


 Later on it will be useful to have the catchment ID in the dataframe, so we add it here:

In [70]:
data['id'] = id


We can also see that the `discharge_vol` column contains `NaN` values - these usually indicate missing data. There are various things we can do to handle (or impute) missing values, but for now let's just remove them: 

In [71]:
data = data.dropna(subset=['discharge_vol'])
data.head()

,date,precipitation,pet,temperature,discharge_spec,discharge_vol,peti,humidity,shortwave_rad,longwave_rad,windspeed,id
457,1972-01-01,0.00,0.21,3.92,1.12,5.37,0.21,4.20,15.49,284.96,4.77,97002
458,1972-01-02,0.84,0.20,3.66,1.05,5.03,0.32,4.53,8.64,311.41,5.63,97002
459,1972-01-03,2.09,0.31,3.69,1.08,5.20,0.44,4.25,8.75,310.61,4.49,97002
460,1972-01-04,1.38,0.01,3.09,1.19,5.73,0.01,4.34,5.78,308.46,2.29,97002
461,1972-01-05,0.47,0.02,2.61,1.12,5.37,0.03,4.03,9.34,304.41,1.72,97002


 Recall the water balance equation from lecture 1:

 $\frac{dS}{dt} = P - E - Q$

 where $\frac{dS}{dt}$ is the change in storage over time, $P$ is precipitation, $E$ is evaporation and $Q$ is streamflow. Also recall that over long time periods we can assume the storage term tends towards zero. Now we can write:

 $0 = P - E - Q$

 and hence:

 $E = P - Q.$

 This is convenient because evaporation is hard to measure accurately. Let's use the equation above to estimate the catchment-averaged evaporation. We will work at annual timescales so that we can reasonably assume that the storage term is negligible. First we need to compute the annual precipitation and discharge. To do this we typically use the "water year" instead of the calendar year. This avoids the potential for large errors in the water balance because catchment storage can vary significantly during the wet season. In the UK the water year is taken as 1st October to 30th September. Fortunately Pandas has some magic that allows us to easily aggregate by water year:

In [72]:
data['water_year'] = data['date'].dt.to_period('Y-SEP')
data.head()


,date,precipitation,pet,temperature,discharge_spec,discharge_vol,peti,humidity,shortwave_rad,longwave_rad,windspeed,id,water_year
457,1972-01-01,0.00,0.21,3.92,1.12,5.37,0.21,4.20,15.49,284.96,4.77,97002,1972
458,1972-01-02,0.84,0.20,3.66,1.05,5.03,0.32,4.53,8.64,311.41,5.63,97002,1972
459,1972-01-03,2.09,0.31,3.69,1.08,5.20,0.44,4.25,8.75,310.61,4.49,97002,1972
460,1972-01-04,1.38,0.01,3.09,1.19,5.73,0.01,4.34,5.78,308.46,2.29,97002,1972
461,1972-01-05,0.47,0.02,2.61,1.12,5.37,0.03,4.03,9.34,304.41,1.72,97002,1972


 Here, `A-SEP` is a period alias for "annual frequency, anchored end of September". Learn more about period aliases by consulting the [Pandas documentation](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-period-aliases).



 We also need to convert the discharge from m3/s to m3/day:

 Now we group our dataframe by the new `water_year` column, and compute the sum of precipitation and discharge:

In [73]:
data['discharge_vol'] *= 60 * 60 * 24
data = data.groupby(['id', 'water_year'])[['precipitation', 'pet', 'discharge_spec', 'discharge_vol']].sum().reset_index()


 Aggregating data is an extremely useful skill in hydrology. Think about how you might use Pandas to aggregate by month or by season.



 When making comparisons between catchments, it is common to transform all variables to a *depth* so that the effect of catchment area is reduced. This allows us to compare the hydrological behaviour of a large catchment (e.g. Tweed) with a much smaller catchment. Let's load the catchment attributes and find the area of our catchment.

In [74]:
metadata = pd.read_csv(os.path.join(DATADIR, f'CAMELS_GB_topographic_attributes.csv'))
metadata['gauge_id'] = metadata['gauge_id'].astype(str)
area = metadata[metadata['gauge_id'] == id]['area'].values[0]
area *= 1e6 # km2 -> m2 


 Now we can transform our data to mm/day:

In [75]:
data['discharge_vol'] /= area # m3 -> m
data['discharge_vol'] *= 1000 # m -> mm


 If you look at the dataframe, you see that column `discharge_vol` is now the same as `discharge_spec`. In future, you can use `discharge_spec` directly, without the need for transformation. We now have everything we need to estimate evaporation using the water balance equation:

In [76]:
data['evaporation'] = data['precipitation'] - data['discharge_vol']


 Let's plot this data:

In [78]:
import matplotlib.pyplot as plt

data = data.set_index('water_year')
data.plot(y=['precipitation', 'discharge_spec', 'evaporation'], figsize=(12, 6))

plt.title(f'Water balance for catchment {id}')
plt.xlabel('Water year end')
plt.ylabel('Depth (mm)')
plt.legend(['Precipitation', 'Discharge', 'Evaporation'])
plt.grid(True)
plt.show()

data = data.reset_index()

KeyError: "None of ['water_year'] are in the columns"

 ## Land cover impacts

 We will cover the drivers of evaporation in more detail later on the course. One question we may have is the role of different land cover types on the water balance. Let's investigate whether land use impacts evaporation by looking at some forested catchments:

In [ ]:
metadata_lu = pd.read_csv(os.path.join(DATADIR, f'CAMELS_GB_landcover_attributes.csv'))


 Have a look at the columns in `metadata_lu` and consult Coxon et al. (2020). Which columns represent forest? Create a new column called `forest_perc` that combines the two types.

In [ ]:
raise NotImplementedError()


 Now identify catchments with more than 20% forest:

In [ ]:
raise NotImplementedError()


 To compare the impact of vegetation on runoff generation, it would be useful to compute a summary measure for each catchment. One such measure, or signature, is the runoff ratio, defined as the proportion of precipitation that becomes runoff. We can calculate this as follows:

In [ ]:
data_sum = data.groupby('id')[['precipitation', 'discharge_vol']].sum()


 Now we can calculate the runoff ratio:

In [ ]:
data_sum['runoff_ratio'] = data_sum['discharge_vol'] / data_sum['precipitation']


 ## Optional exercise

 Compute the runoff ratio for every catchment with greater than 20% forest (HINT: write a loop to perform the necessary steps). Using `statsmodels`, fit a linear regression model to the data to test the hypothesis that runoff ratio is related to the forest extent.